# Feature Engineering y data enrichment

## Enriquecimiento del dataset

### Descargar datos desde api 

Instalamos librerías necesarias para Open-Meteo

In [3]:
!pip install openmeteo-requests requests-cache retry-requests

   ---------------------------------------- 0.0/670.9 kB ? eta -:--:--
   ---------------------------------------- 670.9/670.9 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 4.2 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
Importamos librerias y datos necesarios

In [1]:
import pandas as pd

# --- 1. Carga del Dataset Base ---
ruta_dataset = "../data/processed/dataset_final_etiquetado.csv"
df = pd.read_csv(ruta_dataset)
df['date'] = pd.to_datetime(df['date']) # Muy importante para manipular fechas

print("✅ Dataset base cargado exitosamente.")
print(f"Dimensiones iniciales: {df.shape}")

# --- 2. Ingeniería de Características (Feature Engineering) ---

# --- Características Temporales ---
df['año'] = df['date'].dt.year
df['mes'] = df['date'].dt.month
df['dia_del_año'] = df['date'].dt.dayofyear # El número de día del 1 al 365

# --- Característica de Interacción ---
df['rango_temp_diario'] = df['TMAX'] - df['TMIN']

print("\n✅ Nuevas características creadas:")
# Mostramos el resultado con las nuevas columnas al final
display(df[['date', 'año', 'mes', 'dia_del_año', 'rango_temp_diario', 'granizo']].head())

✅ Dataset base cargado exitosamente.
Dimensiones iniciales: (37399, 8)

✅ Nuevas características creadas:


,date,año,mes,dia_del_año,rango_temp_diario,granizo
0,2000-01-01,2000,1,1,NaN,0
1,2000-01-01,2000,1,1,15.8,1
2,2000-01-01,2000,1,1,NaN,0
3,2000-01-01,2000,1,1,15.8,0
4,2000-01-02,2000,1,2,17.5,0


Cargamos el dataset y mostramos unas nuevas características agregadas, día del año y rango de temperatura diario.
La intención es tener la mayor cantidad de datos disponibles para un análisis más exhaustivo y ver que patrones podemos encontrar.

Veremos el resultado del nuevo df

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37399 entries, 0 to 37398
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               37399 non-null  datetime64[ns]
 1   station_name       37399 non-null  object        
 2   PRCP               9668 non-null   float64       
 3   SNWD               225 non-null    float64       
 4   TAVG               37397 non-null  float64       
 5   TMAX               19599 non-null  float64       
 6   TMIN               35148 non-null  float64       
 7   granizo            37399 non-null  int64         
 8   año                37399 non-null  int32         
 9   mes                37399 non-null  int32         
 10  dia_del_año        37399 non-null  int32         
 11  rango_temp_diario  18324 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int32(3), int64(1), object(1)
memory usage: 3.0+ MB


Prueba de api - Comparación de estaciones vs. obtenidas por API

In [7]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# --- 1. Configuración ---
# (Asumimos que el cliente de Open-Meteo ya está configurado en una celda anterior)
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# --- 2. Nuestros datos de estaciones (ahora con la elevación conocida) ---
stations_info_data = {
    'station_name': [
        'MENDOZA AERO, AR', 'SAN MARTIN, AR', 'MENDOZA OBSERVATORIO, AR', 
        'MALARGUE, AR', 'SAN RAFAEL, AR'
    ],
    'latitude': [-32.83, -33.08, -32.9, -35.48, -34.58],
    'longitude': [-68.78, -68.48, -68.87, -69.58, -68.33],
    'elevation_conocida': [704, 653, 827, 1425, 748] # Elevación en metros
}
stations_info_df = pd.DataFrame(stations_info_data)

print("--- Verificando Coordenadas con la API de Open-Meteo ---")
print("Estación                       | Elevación Conocida | Elevación API (Modelo)")
print("-------------------------------|--------------------|------------------------")

# --- 3. Bucle de Verificación ---
for index, station in stations_info_df.iterrows():
    params = {
        "latitude": station['latitude'],
        "longitude": station['longitude'],
        "daily": "temperature_2m_max", # Pedimos cualquier variable para obtener la metadata
        "start_date": "2024-01-01",
        "end_date": "2024-01-01"
    }
    
    try:
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
        response = responses[0]
        
        # Obtenemos la elevación de la respuesta de la API
        elevation_api = response.Elevation()
        
        # Imprimimos la comparación
        print(f"{station['station_name']:<30} | {station['elevation_conocida']:>18}m | {elevation_api:.2f}m")
        
    except Exception as e:
        print(f"Error al verificar {station['station_name']}: {e}")

--- Verificando Coordenadas con la API de Open-Meteo ---
Estación                       | Elevación Conocida | Elevación API (Modelo)
-------------------------------|--------------------|------------------------
MENDOZA AERO, AR               |                704m | 701.00m
SAN MARTIN, AR                 |                653m | 661.00m
MENDOZA OBSERVATORIO, AR       |                827m | 830.00m
MALARGUE, AR                   |               1425m | 1418.00m
SAN RAFAEL, AR                 |                748m | 691.00m


Descarga unitaria de estaciones(Para descargar en su pc, solo correr esto, lo anterior fue de prueba y conocimiento de errores)

In [32]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import time
import os
import calendar

# --- 1. Definimos Nuestras Estaciones Objetivo ---
stations_info_data = {
    'station_name': ['MENDOZA AERO, AR', 'SAN MARTIN, AR', 'MENDOZA OBSERVATORIO, AR', 'MALARGUE, AR', 'SAN RAFAEL, AR'],
    'latitude': [-32.83, -33.08, -32.9, -35.48, -34.58],
    'longitude': [-68.78, -68.48, -68.87, -69.58, -68.33]
}
stations_info_df = pd.DataFrame(stations_info_data)
print("✅ Lista de estaciones definida.")

# --- 2. Configurar la API de Open-Meteo ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)
print("✅ Cliente de API listo.")

# --- 3. Función de Descarga (Versión Adaptativa y Corregida) ---
def descargar_openmeteo_estacion(station_info):
    lat, lon = station_info['latitude'], station_info['longitude']
    start_year, end_year = 2000, 2024
    
    lista_df_obtenidos = []
    print(f"  -> Procesando años desde {start_year} hasta {end_year}...")
    
    for year in range(start_year, end_year + 1):
        daily_vars = ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_mean", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration", "dew_point_2m_mean", "relative_humidity_2m_mean", "pressure_msl_mean"]
        try:
            # Intento 1: Pedir el año completo
            params_anual = {"latitude": lat, "longitude": lon, "start_date": f"{year}-01-01", "end_date": f"{year}-12-31", "daily": daily_vars, "timezone": "auto"}
            responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params_anual, timeout=120)
            response = responses[0]
            
            daily = response.Daily()
            daily_data = {"date": pd.date_range(start=pd.to_datetime(daily.Time(), unit="s"), end=pd.to_datetime(daily.TimeEnd(), unit="s"), freq=pd.Timedelta(seconds=daily.Interval()), inclusive="left")}
            for i, var in enumerate(daily_vars): daily_data[f"om_{var}"] = daily.Variables(i).ValuesAsNumpy()
            
            lista_df_obtenidos.append(pd.DataFrame(data=daily_data))
            print(f"    -> Datos para el año {year} obtenidos de una vez.")
            time.sleep(5)
        except Exception as e_year:
            # Si falla el intento anual, reintentamos mes por mes
            print(f"    -> ERROR al obtener el año {year} completo. Reintentando mes por mes...")
            for month in range(1, 13):
                try:
                    ultimo_dia = calendar.monthrange(year, month)[1]
                    params_mensual = {"latitude": lat, "longitude": lon, "start_date": f"{year}-{month:02d}-01", "end_date": f"{year}-{month:02d}-{ultimo_dia}", "daily": daily_vars, "timezone": "auto"}
                    responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params_mensual, timeout=60)
                    response = responses[0]

                    daily = response.Daily()
                    daily_data = {"date": pd.date_range(start=pd.to_datetime(daily.Time(), unit="s"), end=pd.to_datetime(daily.TimeEnd(), unit="s"), freq=pd.Timedelta(seconds=daily.Interval()), inclusive="left")}
                    for i, var in enumerate(daily_vars): daily_data[f"om_{var}"] = daily.Variables(i).ValuesAsNumpy()
                    
                    lista_df_obtenidos.append(pd.DataFrame(data=daily_data))
                    print(f"      -> Mes {month}/{year} OK.")
                    time.sleep(2)
                except Exception as e_month:
                    print(f"      -> ERROR en mes {month}/{year}. Error: {e_month}")

    if lista_df_obtenidos:
        return pd.concat(lista_df_obtenidos, ignore_index=True)
    else:
        return pd.DataFrame()

# --- 4. Bucle Principal de Descarga ---
print("\nIniciando descarga de datos de Open-Meteo...")
os.makedirs("../data/raw/openmeteo/", exist_ok=True) 

for index, station in stations_info_df.iterrows():
    station_name = station['station_name']
    nombre_archivo_checkpoint = f"../data/raw/openmeteo/{station_name.replace(', AR', '').replace(' ', '_')}.csv"
    
    if os.path.exists(nombre_archivo_checkpoint):
        print(f"\nCheckpoint encontrado para '{station_name}'. Saltando.")
        continue

    print(f"\nDescargando datos para '{station_name}'...")
    df_openmeteo = descargar_openmeteo_estacion(station)
    
    if not df_openmeteo.empty:
        df_openmeteo['station_name'] = station_name
        df_openmeteo['latitude'] = station['latitude']
        df_openmeteo['longitude'] = station['longitude']
        df_openmeteo.to_csv(nombre_archivo_checkpoint, index=False)
        print(f"✅ Checkpoint guardado para '{station_name}'")

✅ Lista de estaciones definida.
✅ Cliente de API listo.

Iniciando descarga de datos de Open-Meteo...

Descargando datos para 'MENDOZA AERO, AR'...
  -> Procesando años desde 2000 hasta 2024...
    -> Datos para el año 2000 obtenidos de una vez.
    -> Datos para el año 2001 obtenidos de una vez.
    -> Datos para el año 2002 obtenidos de una vez.
    -> Datos para el año 2003 obtenidos de una vez.
    -> Datos para el año 2004 obtenidos de una vez.
    -> Datos para el año 2005 obtenidos de una vez.
    -> Datos para el año 2006 obtenidos de una vez.
    -> Datos para el año 2007 obtenidos de una vez.
    -> Datos para el año 2008 obtenidos de una vez.
    -> Datos para el año 2009 obtenidos de una vez.
    -> Datos para el año 2010 obtenidos de una vez.
    -> Datos para el año 2011 obtenidos de una vez.
    -> Datos para el año 2012 obtenidos de una vez.
    -> Datos para el año 2013 obtenidos de una vez.
    -> Datos para el año 2014 obtenidos de una vez.
    -> Datos para el año 2

### Unión, limpieza y transformación de datos

Luego de ver que sucedieron errores en la descarga y luego la unión se descarga nuevamente los datos, pero solamente por estación sin unir o demás.